In [3]:
import os
import torchaudio
import pandas as pd
import numpy as np
import torch
import torchaudio.transforms as T 
from tqdm.notebook import tqdm
import warnings
import gc
from datetime import datetime
import pywt
from scipy import signal
from scipy.signal import butter, filtfilt
import warnings
warnings.filterwarnings('ignore')

# ==================== ПАРАМЕТРЫ ДЛЯ ВЕЙВЛЕТ-ПРИЗНАКОВ ====================
WIN_LEN = 20      # 20 кадров = 0.20 сек
HOP_LEN = 12      # 12 кадров (перекрытие 40%)
N_WAVELET_LEVELS = 8  # 8 уровней вейвлет-разложения
TARGET_FREQ_MIN = 1800  # Минимальная частота диапазона
TARGET_FREQ_MAX = 3200  # Максимальная частота диапазона
SAMPLE_RATE = 16000
HOP_LENGTH = 160  # 160 сэмплов = 10 мс при 16 кГц

# Расчет общего количества признаков
# Для каждого уровня: 2^(уровень) поддиапазонов, берем 3 статистики
WAVELET_FEATURES_PER_LEVEL = 3  # средняя энергия, std, энтропия
TOTAL_WAVELET_FEATURES = N_WAVELET_LEVELS * WAVELET_FEATURES_PER_LEVEL  # 8 * 3 = 24

# Валидные типы неисправностей
VALID_FAULTS = {'MF1', 'MF2', 'MF3', 'MF4', 'PC1', 'PC2', 'PC3', 'PC4', 'N'}

print("⚙️  ПАРАМЕТРЫ ОБРАБОТКИ ВЕЙВЛЕТ:")
print(f"   win_len: {WIN_LEN} кадров ({WIN_LEN*0.01:.2f} сек)")
print(f"   hop_len: {HOP_LEN} кадров")
print(f"   Перекрытие: {(1 - HOP_LEN/WIN_LEN)*100:.1f}%")
print(f"   Уровней вейвлет: {N_WAVELET_LEVELS}")
print(f"   Целевой диапазон: {TARGET_FREQ_MIN}-{TARGET_FREQ_MAX} Гц")
print(f"   Признаков на окно: {TOTAL_WAVELET_FEATURES}")

# Функция для полосовой фильтрации (1800-3200 Гц)
def bandpass_filter(signal_data, lowcut=TARGET_FREQ_MIN, highcut=TARGET_FREQ_MAX, fs=SAMPLE_RATE, order=4):
    """Полосовой фильтр Баттерворта для выделения целевого диапазона"""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, signal_data)
    return y

# Функция для извлечения вейвлет-признаков
def extract_wavelet_features(audio_signal, levels=N_WAVELET_LEVELS, wavelet='db4'):
    """
    Извлекает вейвлет-признаки из аудиосигнала
    Возвращает признаки для каждого окна
    """
    # Применяем полосовой фильтр
    filtered_audio = bandpass_filter(audio_signal)
    
    # Выполняем вейвлет-разложение
    coeffs = pywt.wavedec(filtered_audio, wavelet, level=levels)
    
    features = []
    
    # Для каждого уровня вейвлет-разложения вычисляем статистики
    for i, coeff in enumerate(coeffs):
        if len(coeff) > 0:
            # 1. Средняя энергия
            energy = np.mean(coeff**2)
            
            # 2. Стандартное отклонение
            std_dev = np.std(coeff)
            
            # 3. Энтропия Шеннона
            coeff_normalized = np.abs(coeff) / (np.sum(np.abs(coeff)) + 1e-10)
            entropy = -np.sum(coeff_normalized * np.log2(coeff_normalized + 1e-10))
            
            features.extend([energy, std_dev, entropy])
        else:
            features.extend([0, 0, 0])
    
    # Обеспечиваем фиксированное количество признаков
    expected_features = levels * 3
    if len(features) < expected_features:
        features.extend([0] * (expected_features - len(features)))
    elif len(features) > expected_features:
        features = features[:expected_features]
    
    return np.array(features, dtype=np.float32)

def create_output_structure(audio_folder, config_name):
    """Создает структуру папок для выходных данных"""
    output_dir = os.path.join(audio_folder, f"features_{config_name}")
    
    dirs_to_create = [
        output_dir,
        os.path.join(output_dir, "by_folder"),
        os.path.join(output_dir, "combined"),
        os.path.join(output_dir, "logs")
    ]
    
    for directory in dirs_to_create:
        os.makedirs(directory, exist_ok=True)
        print(f"📁 Создана папка: {directory}")
    
    return output_dir

def generate_config_name():
    """Генерирует понятное имя конфигурации"""
    return f"win{WIN_LEN}_hop{HOP_LEN}_wavelet{N_WAVELET_LEVELS}_feat{TOTAL_WAVELET_FEATURES}"

def safe_extract_wavelet_features(audio_file, log_fn):
    """Извлекает вейвлет-признаки из аудиофайла с обработкой ошибок"""
    try:
        # Загружаем аудио
        waveform, sr = torchaudio.load(audio_file, normalize=True)
        
        # Ресемплируем при необходимости
        if sr != SAMPLE_RATE:
            resampler = T.Resample(orig_freq=sr, new_freq=SAMPLE_RATE)
            waveform = resampler(waveform)
        
        # Стерео -> моно
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        # Конвертируем в numpy
        audio_signal = waveform.squeeze().numpy()
        
        # Длина окна в сэмплах
        win_length_samples = WIN_LEN * HOP_LENGTH  # 20 * 160 = 3200 сэмплов
        hop_length_samples = HOP_LEN * HOP_LENGTH  # 12 * 160 = 1920 сэмплов
        
        # Разбиваем на окна
        windows = []
        start = 0
        
        while start + win_length_samples <= len(audio_signal):
            window_signal = audio_signal[start:start + win_length_samples]
            
            # Извлекаем вейвлет-признаки для окна
            features = extract_wavelet_features(window_signal)
            windows.append(features)
            
            start += hop_length_samples
        
        # Если файл слишком короткий
        if not windows and len(audio_signal) > 0:
            window_signal = audio_signal[:min(len(audio_signal), win_length_samples)]
            if len(window_signal) < win_length_samples:
                window_signal = np.pad(window_signal, (0, win_length_samples - len(window_signal)), 'constant')
            features = extract_wavelet_features(window_signal)
            windows.append(features)
        
        return np.array(windows) if windows else np.array([]).reshape(0, TOTAL_WAVELET_FEATURES)
        
    except Exception as e:
        log_fn(f"❌ Ошибка извлечения вейвлет-признаков из {os.path.basename(audio_file)}: {e}", "ERROR")
        return None

def parse_and_validate_filename(filename, log_fn):
    """Парсит имя файла и валидирует тип неисправности"""
    basename = filename.replace('.wav', '')
    parts = basename.split('_')
    
    model_type = 'unknown'
    maneuvering_direction = 'unknown'
    fault = 'unknown'
    file_info = {
        'original_name': filename,
        'parts_count': len(parts),
        'is_valid': False,
        'validation_notes': []
    }
    
    if len(parts) >= 3:
        model_type = parts[0]
        maneuvering_direction = parts[1]
        fault_candidate = parts[2]
        
        if fault_candidate in VALID_FAULTS:
            fault = fault_candidate
            file_info['is_valid'] = True
        else:
            found = False
            for i, part in enumerate(parts[2:], start=2):
                if part in VALID_FAULTS:
                    fault = part
                    file_info['validation_notes'].append(f"Найден fault в позиции {i}: {part}")
                    file_info['is_valid'] = True
                    found = True
                    break
            
            if not found:
                if fault_candidate.isdigit():
                    file_info['validation_notes'].append(f"Fault похож на число: {fault_candidate}")
                    fault = 'unknown'
                else:
                    fault = fault_candidate
                    file_info['validation_notes'].append(f"Неизвестный тип fault: {fault_candidate}")
    
    else:
        file_info['validation_notes'].append(f"Необычная структура имени: только {len(parts)} частей")
    
    if not file_info['is_valid']:
        log_fn(f"⚠️  Проблема с файлом {filename}: {file_info['validation_notes']}", "WARNING")
    
    return model_type, maneuvering_direction, fault, file_info

def process_audio_file_wavelet(audio_path, log_fn):
    """Обрабатывает один аудиофайл для вейвлет-признаков"""
    filename = os.path.basename(audio_path)
    
    # 1. Валидация файла
    is_valid, validation_msg = validate_audio_file(audio_path)
    if not is_valid:
        log_fn(f"❌ Файл {filename} не прошел валидацию: {validation_msg}", "ERROR")
        return None, 0, False
    
    # 2. Извлечение вейвлет-признаков
    windows = safe_extract_wavelet_features(audio_path, log_fn)
    if windows is None or len(windows) == 0:
        return None, 0, False
    
    num_windows = len(windows)
    
    # 3. Парсинг имени файла
    model_type, maneuvering_direction, fault, file_info = parse_and_validate_filename(filename, log_fn)
    
    # Создаем DataFrame для окон
    windows_df = pd.DataFrame(windows)
    
    # Переименовываем колонки для признаков
    num_features = windows_df.shape[1]
    windows_df.columns = [f'wavelet_{i:03d}' for i in range(num_features)]
    
    # Добавляем метаданные
    windows_df['model_type'] = model_type
    windows_df['maneuvering_direction'] = maneuvering_direction
    windows_df['fault'] = fault
    windows_df['filename'] = filename
    windows_df['file_valid'] = file_info['is_valid']
    
    # Проверяем количество признаков
    if num_features != TOTAL_WAVELET_FEATURES:
        log_fn(f"⚠️  Неожиданное количество вейвлет-признаков в {filename}: {num_features} (ожидалось {TOTAL_WAVELET_FEATURES})", "WARNING")
    
    return windows_df, num_windows, True

def validate_audio_file(audio_path):
    """Проверяет аудиофайл перед обработкой"""
    try:
        if not os.path.exists(audio_path):
            return False, "Файл не существует"
        
        if os.path.getsize(audio_path) < 1024:
            return False, "Файл слишком маленький"
        
        metadata = torchaudio.info(audio_path)
        duration = metadata.num_frames / metadata.sample_rate
        
        if duration < 0.4 or duration > 0.6:
            return False, f"Неправильная длительность: {duration:.2f} сек"
        
        return True, f"OK ({duration:.2f} сек, {metadata.sample_rate} Гц)"
    
    except Exception as e:
        return False, f"Ошибка проверки: {str(e)}"

def setup_logging(output_dir):
    """Настраивает логирование"""
    log_file = os.path.join(output_dir, "logs", f"wavelet_processing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
    
    def log_message(message, level="INFO"):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] [{level}] {message}"
        print(log_entry)
        
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry + '\n')
    
    return log_message

def main_processing():
    """Основная функция обработки для вейвлет-признаков"""
    audio_folder = r"C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone"
    
    # Генерируем имя конфигурации
    config_name = generate_config_name()
    print(f"\n📊 КОНФИГУРАЦИЯ ОБРАБОТКИ ВЕЙВЛЕТ: {config_name}")
    print(f"   Всего признаков на окно: {TOTAL_WAVELET_FEATURES}")
    
    # Создаем структуру папок
    output_dir = create_output_structure(audio_folder, config_name)
    log_fn = setup_logging(output_dir)
    
    log_fn("🚀 ЗАПУСК ОБРАБОТКИ ВЕЙВЛЕТ-ПРИЗНАКОВ", "INFO")
    log_fn("=" * 60, "INFO")
    
    # Собираем все папки для обработки
    folders_to_process = []
    
    for main_folder in ['A', 'B', 'C']:
        main_path = os.path.join(audio_folder, main_folder)
        if not os.path.exists(main_path):
            log_fn(f"⚠️  Папка {main_path} не найдена, пропускаем", "WARNING")
            continue
        
        for data_split in ['train', 'valid', 'test']:
            split_path = os.path.join(main_path, data_split)
            if not os.path.exists(split_path):
                log_fn(f"⚠️  Папка {split_path} не найдена, пропускаем", "WARNING")
                continue
            
            for mic_folder in ['mic1', 'mic2']:
                mic_path = os.path.join(split_path, mic_folder)
                if not os.path.exists(mic_path):
                    log_fn(f"⚠️  Папка {mic_path} не найдена, пропускаем", "WARNING")
                    continue
                
                folders_to_process.append({
                    'main_folder': main_folder,
                    'data_split': data_split,
                    'mic_folder': mic_folder,
                    'path': mic_path,
                    'id': f"{main_folder}/{data_split}/{mic_folder}"
                })
    
    log_fn(f"📋 Найдено папок для обработки: {len(folders_to_process)}", "INFO")
    
    # Обрабатываем каждую папку
    all_stats = []
    all_dfs = []
    
    for i, folder_info in enumerate(folders_to_process, 1):
        log_fn(f"\n{'='*50}", "INFO")
        log_fn(f"📦 Папка {i}/{len(folders_to_process)}: {folder_info['id']}", "INFO")
        
        stats, folder_df = process_folder_wavelet(
            folder_info['main_folder'],
            folder_info['data_split'],
            folder_info['mic_folder'],
            folder_info['path'],
            output_dir,
            log_fn
        )
        
        if stats:
            all_stats.append(stats)
        
        if folder_df is not None:
            all_dfs.append(folder_df)
    
    # Создаем объединенный файл
    if all_dfs:
        log_fn(f"\n🔄 Создание объединенного файла...", "INFO")
        
        combined_df = pd.concat(all_dfs, ignore_index=True)
        
        # Сохраняем объединенный файл
        combined_path = os.path.join(output_dir, "combined", "all_data_wavelet.parquet")
        combined_df.to_parquet(
            combined_path,
            engine='pyarrow',
            compression='snappy',
            index=False
        )
        
        combined_size = os.path.getsize(combined_path) / (1024**3)  # GB
        log_fn(f"✅ Объединенный файл создан", "INFO")
        log_fn(f"   Всего записей: {len(combined_df):,}", "INFO")
        log_fn(f"   Всего вейвлет-признаков: {TOTAL_WAVELET_FEATURES}", "INFO")
        log_fn(f"   Размер: {combined_size:.2f} GB", "INFO")
        log_fn(f"   Сохранено: {combined_path}", "INFO")
        
        # Генерация отчета
        generate_final_report_wavelet(all_stats, output_dir, log_fn, config_name)

def process_folder_wavelet(main_folder, data_split, mic_folder, folder_path, output_dir, log_fn):
    """Обрабатывает одну папку для вейвлет-признаков"""
    folder_name = f"{main_folder}_{data_split}_{mic_folder}"
    log_fn(f"📁 Начало обработки папки: {folder_name}", "INFO")
    
    audio_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.wav')]
    
    if not audio_files:
        log_fn(f"   ⚠️  Нет WAV файлов в папке", "WARNING")
        return None, None
    
    log_fn(f"   Найдено файлов: {len(audio_files)}", "INFO")
    
    folder_stats = {
        'folder_id': folder_name,
        'total_files': len(audio_files),
        'processed_files': 0,
        'failed_files': 0,
        'total_windows': 0,
        'invalid_faults': 0,
        'file_errors': []
    }
    
    all_windows = []
    
    with tqdm(total=len(audio_files), desc=f"{folder_name}", unit="файл") as pbar:
        for audio_file in audio_files:
            audio_path = os.path.join(folder_path, audio_file)
            
            windows_df, num_windows, success = process_audio_file_wavelet(audio_path, log_fn)
            
            if success and windows_df is not None:
                windows_df['source_folder'] = folder_name
                windows_df['data_split'] = data_split
                windows_df['mic_number'] = mic_folder
                windows_df['drone_model'] = main_folder
                
                all_windows.append(windows_df)
                folder_stats['processed_files'] += 1
                folder_stats['total_windows'] += num_windows
                
                if windows_df['fault'].iloc[0] not in VALID_FAULTS:
                    folder_stats['invalid_faults'] += 1
                
                pbar.set_postfix({
                    'окон': num_windows,
                    'fault': windows_df['fault'].iloc[0],
                    'движение': windows_df['maneuvering_direction'].iloc[0]
                })
            else:
                folder_stats['failed_files'] += 1
                folder_stats['file_errors'].append(audio_file)
                log_fn(f"   Файл {audio_file}: ОШИБКА обработки", "WARNING")
            
            pbar.update(1)
            
            if folder_stats['processed_files'] % 100 == 0:
                gc.collect()
    
    if all_windows:
        result_df = pd.concat(all_windows, ignore_index=True)
        result_df['processing_params'] = f"win_len={WIN_LEN},hop_len={HOP_LEN},wavelet_levels={N_WAVELET_LEVELS}"
        result_df['processing_date'] = datetime.now().strftime("%Y-%m-%d")
        
        output_file = f"{folder_name}_wavelet.parquet"
        output_path = os.path.join(output_dir, "by_folder", output_file)
        
        result_df.to_parquet(
            output_path,
            engine='pyarrow',
            compression='snappy',
            index=False
        )
        
        file_size = os.path.getsize(output_path) / (1024**2)
        folder_stats['output_file'] = output_file
        folder_stats['file_size_mb'] = file_size
        folder_stats['total_rows'] = len(result_df)
        folder_stats['total_columns'] = len(result_df.columns)
        
        log_fn(f"✅ Папка обработана: {folder_name}", "INFO")
        log_fn(f"   Успешно: {folder_stats['processed_files']}/{folder_stats['total_files']} файлов", "INFO")
        log_fn(f"   Всего окон: {folder_stats['total_windows']}", "INFO")
        log_fn(f"   Невалидных fault: {folder_stats['invalid_faults']}", "INFO")
        log_fn(f"   Размер файла: {file_size:.2f} MB", "INFO")
        
        return folder_stats, result_df
    else:
        log_fn(f"❌ Нет данных для сохранения в папке {folder_name}", "ERROR")
        return folder_stats, None

def generate_final_report_wavelet(all_stats, output_dir, log_fn, config_name):
    """Генерирует финальный отчет"""
    log_fn(f"\n{'='*60}", "INFO")
    log_fn("📊 ФИНАЛЬНЫЙ ОТЧЕТ ВЕЙВЛЕТ", "INFO")
    log_fn(f"{'='*60}", "INFO")
    
    if not all_stats:
        log_fn("❌ Нет статистики для отчета", "ERROR")
        return
    
    stats_df = pd.DataFrame(all_stats)
    
    total_files = stats_df['total_files'].sum()
    processed_files = stats_df['processed_files'].sum()
    failed_files = stats_df['failed_files'].sum()
    total_windows = stats_df['total_windows'].sum()
    
    log_fn(f"📈 ОБЩАЯ СТАТИСТИКА:", "INFO")
    log_fn(f"   Конфигурация: {config_name}", "INFO")
    log_fn(f"   Всего вейвлет-признаков: {TOTAL_WAVELET_FEATURES}", "INFO")
    log_fn(f"   Обработано папок: {len(all_stats)}", "INFO")
    log_fn(f"   Всего файлов: {total_files:,}", "INFO")
    log_fn(f"   Успешно обработано: {processed_files:,} ({processed_files/total_files*100:.1f}%)", "INFO")
    log_fn(f"   Ошибок обработки: {failed_files:,} ({failed_files/total_files*100:.1f}%)", "INFO")
    log_fn(f"   Всего окон (примеров): {total_windows:,}", "INFO")
    
    stats_csv_path = os.path.join(output_dir, "logs", "wavelet_processing_stats.csv")
    stats_df.to_csv(stats_csv_path, index=False, encoding='utf-8')
    log_fn(f"💾 Статистика сохранена: {stats_csv_path}", "INFO")
    
    print(f"\n{'='*60}")
    print("🎯 РЕЗУЛЬТАТЫ ОБРАБОТКИ ВЕЙВЛЕТ:")
    print(f"{'='*60}")
    print(f"📊 Конфигурация: {config_name}")
    print(f"🔢 Вейвлет-признаков на окно: {TOTAL_WAVELET_FEATURES}")
    print(f"📈 Всего примеров (окон): {total_windows:,}")
    print(f"📁 Результаты сохранены в: {output_dir}")
    print(f"{'='*60}")

# ==================== ЗАПУСК ====================
if __name__ == "__main__":
    print("🚀 ПОДГОТОВКА К ОБРАБОТКЕ ВЕЙВЛЕТ-ПРИЗНАКОВ")
    print("=" * 60)
    print(f"⚙️  ПАРАМЕТРЫ:")
    print(f"   WIN_LEN: {WIN_LEN} (окно 0.{WIN_LEN} сек)")
    print(f"   HOP_LEN: {HOP_LEN} (перекрытие {(1 - HOP_LEN/WIN_LEN)*100:.1f}%)")
    print(f"   УРОВНЕЙ ВЕЙВЛЕТ: {N_WAVELET_LEVELS}")
    print(f"   ЦЕЛЕВОЙ ДИАПАЗОН: {TARGET_FREQ_MIN}-{TARGET_FREQ_MAX} Гц")
    print(f"   ПРИЗНАКОВ НА ОКНО: {TOTAL_WAVELET_FEATURES}")
    print("=" * 60)
    
    # Проверяем наличие необходимых библиотек
    required_libs = ['pyarrow', 'tqdm', 'pywt', 'scipy']
    for lib in required_libs:
        try:
            __import__(lib)
            print(f"✅ {lib} установлен")
        except ImportError:
            print(f"❌ {lib} не установлен. Установите: pip install {lib}")
            exit(1)
    
    print("=" * 60)
    
    # Запускаем обработку
    main_processing()

⚙️  ПАРАМЕТРЫ ОБРАБОТКИ ВЕЙВЛЕТ:
   win_len: 20 кадров (0.20 сек)
   hop_len: 12 кадров
   Перекрытие: 40.0%
   Уровней вейвлет: 8
   Целевой диапазон: 1800-3200 Гц
   Признаков на окно: 24
🚀 ПОДГОТОВКА К ОБРАБОТКЕ ВЕЙВЛЕТ-ПРИЗНАКОВ
⚙️  ПАРАМЕТРЫ:
   WIN_LEN: 20 (окно 0.20 сек)
   HOP_LEN: 12 (перекрытие 40.0%)
   УРОВНЕЙ ВЕЙВЛЕТ: 8
   ЦЕЛЕВОЙ ДИАПАЗОН: 1800-3200 Гц
   ПРИЗНАКОВ НА ОКНО: 24
✅ pyarrow установлен
✅ tqdm установлен
✅ pywt установлен
✅ scipy установлен

📊 КОНФИГУРАЦИЯ ОБРАБОТКИ ВЕЙВЛЕТ: win20_hop12_wavelet8_feat24
   Всего признаков на окно: 24
📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\features_win20_hop12_wavelet8_feat24
📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\features_win20_hop12_wavelet8_feat24\by_folder
📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\features_win20_hop12_wavelet8_feat24\combined
📁 Создана папка: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\features_win20_hop12_wavelet8_feat24\l

A_train_mic1:   0%|          | 0/32400 [00:00<?, ?файл/s]

[2026-01-12 10:42:52] [INFO] ✅ Папка обработана: A_train_mic1
[2026-01-12 10:42:52] [INFO]    Успешно: 32400/32400 файлов
[2026-01-12 10:42:52] [INFO]    Всего окон: 97200
[2026-01-12 10:42:52] [INFO]    Невалидных fault: 0
[2026-01-12 10:42:52] [INFO]    Размер файла: 14.38 MB
[2026-01-12 10:42:53] [INFO] 
[2026-01-12 10:42:53] [INFO] 📦 Папка 2/18: A/train/mic2
[2026-01-12 10:42:53] [INFO] 📁 Начало обработки папки: A_train_mic2
[2026-01-12 10:42:55] [INFO]    Найдено файлов: 32400


A_train_mic2:   0%|          | 0/32400 [00:00<?, ?файл/s]

[2026-01-12 11:02:44] [INFO] ✅ Папка обработана: A_train_mic2
[2026-01-12 11:02:44] [INFO]    Успешно: 32400/32400 файлов
[2026-01-12 11:02:44] [INFO]    Всего окон: 97200
[2026-01-12 11:02:44] [INFO]    Невалидных fault: 0
[2026-01-12 11:02:44] [INFO]    Размер файла: 14.39 MB
[2026-01-12 11:02:44] [INFO] 
[2026-01-12 11:02:44] [INFO] 📦 Папка 3/18: A/valid/mic1
[2026-01-12 11:02:44] [INFO] 📁 Начало обработки папки: A_valid_mic1
[2026-01-12 11:02:45] [INFO]    Найдено файлов: 10800


...

[2026-01-12 14:12:00] [INFO] ✅ Папка обработана: C_valid_mic2
[2026-01-12 14:12:00] [INFO]    Успешно: 10800/10800 файлов
[2026-01-12 14:12:00] [INFO]    Всего окон: 32400
[2026-01-12 14:12:00] [INFO]    Невалидных fault: 0
[2026-01-12 14:12:00] [INFO]    Размер файла: 4.67 MB
[2026-01-12 14:12:01] [INFO] 
[2026-01-12 14:12:01] [INFO] 📦 Папка 17/18: C/test/mic1
[2026-01-12 14:12:01] [INFO] 📁 Начало обработки папки: C_test_mic1
[2026-01-12 14:12:01] [INFO]    Найдено файлов: 10800


C_test_mic1:   0%|          | 0/10800 [00:00<?, ?файл/s]

[2026-01-12 14:20:32] [INFO] ✅ Папка обработана: C_test_mic1
[2026-01-12 14:20:32] [INFO]    Успешно: 10800/10800 файлов
[2026-01-12 14:20:32] [INFO]    Всего окон: 32400
[2026-01-12 14:20:32] [INFO]    Невалидных fault: 0
[2026-01-12 14:20:32] [INFO]    Размер файла: 4.67 MB
[2026-01-12 14:20:33] [INFO] 
[2026-01-12 14:20:33] [INFO] 📦 Папка 18/18: C/test/mic2
[2026-01-12 14:20:33] [INFO] 📁 Начало обработки папки: C_test_mic2
[2026-01-12 14:20:33] [INFO]    Найдено файлов: 10800


C_test_mic2:   0%|          | 0/10800 [00:00<?, ?файл/s]

[2026-01-12 14:29:06] [INFO] ✅ Папка обработана: C_test_mic2
[2026-01-12 14:29:06] [INFO]    Успешно: 10800/10800 файлов
[2026-01-12 14:29:06] [INFO]    Всего окон: 32400
[2026-01-12 14:29:06] [INFO]    Невалидных fault: 0
[2026-01-12 14:29:06] [INFO]    Размер файла: 4.67 MB
[2026-01-12 14:29:06] [INFO] 
🔄 Создание объединенного файла...
[2026-01-12 14:29:13] [INFO] ✅ Объединенный файл создан
[2026-01-12 14:29:13] [INFO]    Всего записей: 971,721
[2026-01-12 14:29:13] [INFO]    Всего вейвлет-признаков: 24
[2026-01-12 14:29:13] [INFO]    Размер: 0.11 GB
[2026-01-12 14:29:13] [INFO]    Сохранено: C:\Users\lkost\Desktop\Python\vkr_drones_v2\drone\features_win20_hop12_wavelet8_feat24\combined\all_data_wavelet.parquet
[2026-01-12 14:29:13] [INFO] 
[2026-01-12 14:29:13] [INFO] 📊 ФИНАЛЬНЫЙ ОТЧЕТ ВЕЙВЛЕТ
[2026-01-12 14:29:13] [INFO] ============================================================
[2026-01-12 14:29:13] [INFO] 📈 ОБЩАЯ СТАТИСТИКА:
[2026-01-12 14:29:13] [INFO]    Конфигурация: win20_

In [2]:
pip install PyWavelets

   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.2 MB 5.6 MB/s eta 0:00:01
   --------------- ------------------------ 1.6/4.2 MB 7.6 MB/s eta 0:00:01
   ------------------------------------- -- 3.9/4.2 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
